---
## Test self play

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from alphazero.models.network import Network
from alphazero.module.replay import ReplayBuffer
from alphazero.game.config import config_2048
from alphazero.train.self_play import run_self_play
from alphazero.train.training import compute_td_target

In [3]:
config = config_2048()

In [4]:
replay_buffer = ReplayBuffer(config.replay)

In [5]:
network = config.factory.network_factory()

Metal device set to: Apple M1 Pro


2022-11-08 22:34:28.245039: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-08 22:34:28.245150: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
run_self_play(config, network, replay_buffer, 1)

/Users/schalappe/miniforge3/envs/env_rl/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (4, 4)
  logger.warn(


Actor n°1 finish ...
Actor n°2 finish ...


In [7]:
len(replay_buffer._data[0])

292

In [8]:
replay_buffer._data[0][-1]

State(observation=array([[  2,   4,  16,   4],
       [  2, 256,  64,  16],
       [ 64,  16, 128,   8],
       [  4,  32,   8,   2]]), reward=4, discount=0.999, action=3, search_stats=SearchStats(search_policy={1: 12, 3: 8}, search_value=0.3770373432017707))

In [9]:
sample = replay_buffer.sample()

In [10]:
len(sample)

1024

In [11]:
td_targets = compute_td_target(config.replay.td_steps, config.replay.td_lambda, sample)

In [12]:
len(td_targets)

1024

In [13]:
td_targets[0]

[array([[ 2,  2,  0,  0],
        [ 4,  8,  4,  0],
        [64,  2, 16,  0],
        [ 2, 16,  4,  0]]),
 7.3163312267070895,
 [0.4, 0.15, 0.45, 0]]

In [14]:
observations, target_values, target_policies = zip(*td_targets)

In [15]:
observations[0]

array([[ 2,  2,  0,  0],
       [ 4,  8,  4,  0],
       [64,  2, 16,  0],
       [ 2, 16,  4,  0]])

In [16]:
def encode(state):
    obs = state.copy()
    obs = np.reshape(obs, -1)
    obs[obs == 0] = 1
    obs = np.log2(obs)
    obs = obs.astype(int)
    return np.reshape(np.eye(31)[obs], -1)

In [17]:
observations = [encode(obs) for obs in observations]

In [18]:
observations[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [19]:
tensor_obs = tf.stack(observations)

In [20]:
tensor_obs

<tf.Tensor: shape=(1024, 496), dtype=float64, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [21]:
predicts = network.model(tensor_obs, training=False)

In [22]:
len(predicts)

2

In [23]:
predicts

[<tf.Tensor: shape=(1024, 4), dtype=float32, numpy=
 array([[0.26123488, 0.25167587, 0.17025577, 0.3168335 ],
        [0.2716772 , 0.25360712, 0.11222921, 0.3624865 ],
        [0.2881742 , 0.2422152 , 0.11043786, 0.35917273],
        ...,
        [0.30563417, 0.22402717, 0.10214236, 0.3681963 ],
        [0.25844336, 0.2382467 , 0.1410388 , 0.36227113],
        [0.27667114, 0.25207597, 0.14451921, 0.32673365]], dtype=float32)>,
 <tf.Tensor: shape=(1024, 1), dtype=float32, numpy=
 array([[-0.12984462],
        [-0.18343288],
        [-0.20527183],
        ...,
        [-0.18799019],
        [-0.1321177 ],
        [-0.05137035]], dtype=float32)>]